## ML Model for Dynamic Characters

### Imports

In [4]:
import keras
import numpy as np
import random

In [6]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Read Data

In [3]:
# get data 
# img = imageio.imread('../resources/textures/sphereTex.png')
# img = img[:, :, 0] # convert to grayscale
# rows = len(img) # rows
# cols = len(img[0]) # cols

# read data
# data = []
# with open("out.txt", "r") as text_file:
#     line = text_file.read()
#     data = line.split('\n')[:-1]
#     data = [ list(map(float, x.split(','))) for x in data ]
# data = np.array(data)

# # split data
# np.random.shuffle(data)
# partition = int(0.8 * len(data))
# X_train = data[:partition, :-1]
# y_train = data[:partition, -1] / 255.0
# X_test = data[partition:, :-1] 
# y_test = data[partition:, -1] / 255.0

In [22]:
# read data
data = []
with open("../resources/occlusion/data.txt", "r") as text_file:
    line = text_file.read()
    data = line.split('\n')[:-1]
    data = [ list(map(float, x.split(' '))) for x in data ]
data = np.array(data)

# split data
np.random.shuffle(data)
partition = int(0.8 * len(data))
X_train = data[:partition, :-1]
y_train = data[:partition, -1]
X_test = data[partition:, :-1]
y_test = data[partition:, -1]

### Build Model

<b>Inputs:</b> u, v, thetas
<br/><b>Output</b>: occlusion factor

In [48]:
# build model
model = keras.Sequential([
    keras.layers.Dense(1024, input_shape=(3,), activation='tanh'), # hidden
    keras.layers.Dense(512, activation='tanh'), # hidden
    keras.layers.Dense(256, activation='tanh'), # hidden
    keras.layers.Dense(1) # output
])
model.compile(optimizer="rmsprop", loss=keras.losses.MeanSquaredError(), metrics="mse")

In [49]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 1024)              4096      
                                                                 
 dense_25 (Dense)            (None, 512)               524800    
                                                                 
 dense_26 (Dense)            (None, 256)               131328    
                                                                 
 dense_27 (Dense)            (None, 1)                 257       
                                                                 
Total params: 660,481
Trainable params: 660,481
Non-trainable params: 0
_________________________________________________________________


### Set up Tensorboard

### Train Model

In [50]:
# train model
history = model.fit(X_train, y_train, epochs=40, batch_size=64)

Epoch 1/40
313/313 [==============================] - 4s 10ms/step - loss: 0.1113 - mse: 0.1113
Epoch 2/40
313/313 [==============================] - 3s 9ms/step - loss: 0.0115 - mse: 0.0115
Epoch 3/40
313/313 [==============================] - 3s 9ms/step - loss: 0.0110 - mse: 0.0110
Epoch 4/40
313/313 [==============================] - 3s 9ms/step - loss: 0.0109 - mse: 0.0109
Epoch 5/40
313/313 [==============================] - 3s 9ms/step - loss: 0.0108 - mse: 0.0108
Epoch 6/40
313/313 [==============================] - 3s 9ms/step - loss: 0.0108 - mse: 0.0108
Epoch 7/40
313/313 [==============================] - 3s 9ms/step - loss: 0.0108 - mse: 0.0108
Epoch 8/40
313/313 [==============================] - 3s 9ms/step - loss: 0.0108 - mse: 0.0108
Epoch 9/40
313/313 [==============================] - 3s 9ms/step - loss: 0.0107 - mse: 0.0107
Epoch 10/40
313/313 [==============================] - 3s 9ms/step - loss: 0.0104 - mse: 0.0104
Epoch 11/40
262/313 [========================>..

KeyboardInterrupt: 

In [7]:
# # generate texture using model
# dim = 512
# inputs = []
# for r in range(dim,0,-1):
#     for c in range(dim):
#         x = [c/(dim-1), r/(dim-1)]
#         inputs.append(x)
# outputs = model.predict(inputs)
# outputs = [ 255 if y[0] > 1 else 255*y[0] for y in outputs ]

8192/8192 [==============================] - 3s 375us/step


In [8]:
# # save image
# img = Image.new('L', (dim,dim))
# img.putdata(outputs)
# img.save('nnTexture.png')

In [40]:
model.predict([
    [0.94197,0.61064,0],
    [0.85878,0.47439,0],
    [0.86127,0.47565,0],
    [0.84563,0.45853,0],
    [0.84873,0.46279,0],
    [0.92041,0.6823,0],
    [0.91853,0.68184,0]
])

1/1 [==============================] - 0s 161ms/step


In [38]:
wab = model.get_weights()

In [39]:
# manual evaluation 2 hidden layers: input = [u, v, theta]
def manual_eval(input):
    w1 = wab[0]
    b1 = wab[1]
    w2 = wab[2]
    b2 = wab[3]
    w3 = wab[4]
    b3 = wab[5]

    z1 = keras.activations.tanh(np.matmul(np.array(input), w1) + b1)
    z2 = keras.activations.tanh(np.matmul(z1, w2) + b2)
    z3 = keras.activations.tanh(np.matmul(z2, w3) + b3)
    return z3.numpy()[0]

0.019228264243694542

### Save to output file

In [35]:
# save weights and biases as buffer
output_buf = str(len(model.layers)) + "\n" # layer count

for i in range(0, len(wab)-1, 2): # iterate through layers
    weights = wab[i]
    output_buf += str(weights.shape[0]) + " " + str(weights.shape[1]) + "\n" # input size output size
    for neuron in weights: # iterate through
        for edges in neuron:
            output_buf += str(edges) + " "
    for biases in wab[i+1]:
        output_buf += str(biases) + " "
    output_buf = output_buf[:-1] + "\n" # ignore extraneous space and end line
    

In [36]:
with open("../resources/evaluators/model.txt", "w") as text_file:
    text_file.write(output_buf) # add more digits

# Second Model

### Create Model

Inputs: Joint Angles
Outputs: AO values for each vertex

In [1]:
import keras
import numpy as np

In [44]:
# read data
header = []
data = []
with open("../resources/occlusion/dataAbs.txt", "r") as text_file:
    line = text_file.read()
    data = [ list(map(float, x.split(' '))) for x in line.split('\n')[:-1] ]
    header = data[:1][0]
    data = data[1:]
data = np.array(data)

num_joints = int(header[0])
vertex_count = int(header[1])

# split data
np.random.shuffle(data)
partition = int(0.8 * len(data))
X_train = data[:partition, :num_joints]
y_train = data[:partition, num_joints:]
X_test = data[partition:, :num_joints]
y_test = data[partition:, num_joints:]

In [45]:
print(header)

[1.0, 1059.0]


In [79]:
# build model
model = keras.Sequential([
    keras.layers.Dense(1059, input_shape=(num_joints,), activation='tanh'), # hidden
    # keras.layers.Dense(?, activation='tanh'), # hidden
    keras.layers.Dense(vertex_count) # output
])
model.compile(optimizer="rmsprop", loss=keras.losses.MeanSquaredError(), metrics="mse")

In [80]:
# train model
history = model.fit(X_train, y_train, epochs=35, batch_size=64)

Epoch 1/35
2/2 [==============================] - 0s 15ms/step - loss: 0.0286 - mse: 0.0286
Epoch 2/35
2/2 [==============================] - 0s 14ms/step - loss: 0.0146 - mse: 0.0146
Epoch 3/35
2/2 [==============================] - 0s 20ms/step - loss: 0.0084 - mse: 0.0084
Epoch 4/35
2/2 [==============================] - 0s 21ms/step - loss: 0.0065 - mse: 0.0065
Epoch 5/35
2/2 [==============================] - 0s 14ms/step - loss: 0.0057 - mse: 0.0057
Epoch 6/35
2/2 [==============================] - 0s 24ms/step - loss: 0.0057 - mse: 0.0057
Epoch 7/35
2/2 [==============================] - 0s 22ms/step - loss: 0.0057 - mse: 0.0057
Epoch 8/35
2/2 [==============================] - 0s 14ms/step - loss: 0.0056 - mse: 0.0056
Epoch 9/35
2/2 [==============================] - 0s 22ms/step - loss: 0.0050 - mse: 0.0050
Epoch 10/35
2/2 [==============================] - 0s 18ms/step - loss: 0.0050 - mse: 0.0050
Epoch 11/35
2/2 [==============================] - 0s 11ms/step - loss: 0.0036 

In [81]:
wab = model.get_weights()

In [82]:
# save weights and biases as buffer
output_buf = str(len(model.layers)) + "\n" # layer count

for i in range(0, len(wab)-1, 2): # iterate through layers
    weights = wab[i]
    output_buf += str(weights.shape[0]) + " " + str(weights.shape[1]) + "\n" # input size output size
    for neuron in weights: # iterate through
        for edges in neuron:
            output_buf += str(edges) + " "
    for biases in wab[i+1]:
        output_buf += str(biases) + " "
    output_buf = output_buf[:-1] + "\n" # ignore extraneous space and end line

In [83]:
with open("../resources/evaluators/modelAbs.txt", "w") as text_file:
    text_file.write(output_buf) # add more digits

In [90]:
p = model.predict([
    X_test[0]
])


1/1 [==============================] - 0s 53ms/step


array([[ 0.00416847, -0.00844724,  0.00868796, ...,  0.00885363,
        -0.00669841,  0.00817197]])